In [59]:
import nest_asyncio
nest_asyncio.apply()

from sc2 import maps
from sc2.bot_ai import BotAI
from sc2.data import Difficulty, Race
from sc2.ids.ability_id import AbilityId
from sc2.ids.buff_id import BuffId
from sc2.ids.unit_typeid import UnitTypeId
from sc2.main import run_game
from sc2.player import Bot, Computer
import random
import math
import numpy as np; from numpy import linspace, pi

### needs to be a state machine

class BBBot(BotAI):
    # pylint: disable=R0912
    async def on_step(self, iteration):
        if iteration % 5 == 0:
            self.tick5 = True
        else:
            self.tick5 = False
        
        bases = self.townhalls
        num_bases = self.townhalls.amount
        num_workers = self.workers.amount
        if num_bases > 0:
            worker_ratio = num_workers/num_bases
        else:
            worker_ratio = 0

        await self.distribute_workers()
        await self.build_supply()
        await self.build_rax()
        await self.build_marines()
        await self.marine_patrol()

        if worker_ratio < 15:
            await self.build_workers()
        else:
            await self.expand()

    async def build_workers(self):
        for cc in self.townhalls.ready.idle:
            if self.can_afford(UnitTypeId.SCV):
                cc.train(UnitTypeId.SCV)

    async def expand(self):
        if self.townhalls.amount < 5 and self.can_afford(UnitTypeId.COMMANDCENTER):
            await self.expand_now()

    async def build_supply(self):
        ccs = self.townhalls(UnitTypeId.COMMANDCENTER).ready
        if ccs.exists:
            cc = ccs.first
            if self.supply_left < 4 and not self.already_pending(UnitTypeId.SUPPLYDEPOT):
                if self.can_afford(UnitTypeId.SUPPLYDEPOT):
                    await self.build(UnitTypeId.SUPPLYDEPOT, near=cc.position.towards(self.game_info.map_center, 10))

    async def build_rax(self):
        ccs = self.townhalls.ready
        if ccs.exists:
            for cc in ccs:
                if self.structures(UnitTypeId.BARRACKS).closer_than(5, cc).amount < 2 and self.already_pending(UnitTypeId.BARRACKS) < 2:
                    if self.can_afford(UnitTypeId.BARRACKS):
                        await self.build(UnitTypeId.BARRACKS, near=cc.position)

    async def build_marines(self):
        if self.units(UnitTypeId.MARINE).amount < self.townhalls.amount * 30:
            ccs = self.townhalls(UnitTypeId.COMMANDCENTER).ready
            if ccs.exists:
                for cc in ccs:
                    if self.structures(UnitTypeId.BARRACKS).closer_than(20, cc).amount > 0:
                        for rax in self.structures(UnitTypeId.BARRACKS).closer_than(20, cc).ready.idle:
                            if self.can_afford(UnitTypeId.MARINE):
                                rax.train(UnitTypeId.MARINE)

    async def marine_patrol2(self):
        mrns = self.units(UnitTypeId.MARINE)

        if mrns.exists:
            for mr in mrns:
                n8 = mr.position.neighbors8
                next_location = mr.position.towards()
                for n in n8:
                    if not self.in_pathing_grid(n):
                        next_location = next_location.offset(n.direction_vector(mr.position))
                #next_location = next_location.offset([random.randrange(-1,1),random.randrange(-1,1)])
                mr.move(next_location)


    async def marine_patrol(self):
        mrns = self.units(UnitTypeId.MARINE)
        ccs = self.townhalls

        if mrns.exists:
            for mr in mrns:
                if self.enemy_units.in_attack_range_of(mr, 20).amount > 0:
                    nearest_enemy = self.enemy_units.in_attack_range_of(mr, 20).closest_to(mr)
                    mr.attack(nearest_enemy)
                else:
                    if self.tick5:
                        ### boid ai begins here ###
                        mr_facing = mr.facing # float, 0 to 2pi
                        pos = mr.position
                        neighbors = self.units(UnitTypeId.MARINE).closest_n_units(pos, 7)
                        if neighbors.exists:
                            avg_neighbor_heading = mr_facing
                            count = 0
                            for n in neighbors:
                                avg_neighbor_heading += n.facing
                                count += 1
                            avg_neighbor_heading = avg_neighbor_heading / count
                            calc_pos = pos.towards(pos + [math.cos(avg_neighbor_heading),
                                                        math.sin(avg_neighbor_heading)],
                                                        8)
                            nearest_neighbor = neighbors.closest_to(pos)
                            nn_heading = nearest_neighbor.facing
                            d_factor = nearest_neighbor.distance_to_squared(pos) + 1
                            calc_pos = calc_pos.towards(pos - [math.cos(nn_heading),
                                                            math.sin(nn_heading)],
                                                            4/d_factor)
                            calc_pos = calc_pos.towards(neighbors.center, 2)

                        else:
                            calc_pos = pos.towards_with_random_angle(pos + [math.cos(mr_facing),
                                                                            math.sin(mr_facing)])
                        calc_pos = calc_pos.towards_with_random_angle(random.choice(self.enemy_start_locations).position,
                                                                    0.5,
                                                                    (math.pi/8)
                                                                    )
                        if self.in_pathing_grid(calc_pos):
                            mr.move(calc_pos)
                        else:
                            mr.move(calc_pos.towards_with_random_angle(pos,15, (math.pi/8)))





                


def main():
    run_game(
        maps.get("(2)CatalystLE"),
        [Bot(Race.Terran, BBBot()),
         Computer(Race.Terran, Difficulty.Hard)],
        realtime=False,
    )


if __name__ == "__main__":
    main()


2023-02-19 21:52:14.904 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.launched (was None)
2023-02-19 21:52:14.905 | INFO     | sc2.controller:create_game:37 - Creating new game
2023-02-19 21:52:14.905 | INFO     | sc2.controller:create_game:38 - Map:     (2)CatalystLE
2023-02-19 21:52:14.906 | INFO     | sc2.controller:create_game:39 - Players: Bot BBBot(Terran), Computer Hard(Terran, RandomBuild)
2023-02-19 21:52:14.907 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.init_game (was Status.launched)
2023-02-19 21:52:20.409 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.in_game (was None)
2023-02-19 21:52:20.410 | INFO     | sc2.main:_play_game:221 - Player 1 - Bot BBBot(Terran)
2023-02-19 21:54:57.396 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.ended (was Status.in_game)
2023-02-19 21:54:57.397 | INFO     | sc2.main:_play_game:228 - Result for player 1 - Bot BBBot(Terran): Defeat
2023